# NCAA Stats Data Pipeline

This "pipeline" is a notebook used to setup NCAA data in our Databricks sandbox. It's largely used as a workaround since we don't have access to DLT/jobs in our sandbox environment; For now, I'll just run the scripts manually like a peasant, but in real-life this could be converted to
DLT pipelines, jobs, etc

The steps in this notebook:
1. Setup the initial schema for landing NCAA data
1. Load raw data into Databricks
1. Run ETL scripts to cleanup and transform data into a format suitable for analysis

## Setup
Run cells in this section to get your environment setup

In [1]:
# Setup module autoreload
%load_ext autoreload
%autoreload 2

In [2]:
# Load environment variables using dotenv

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Create a Spark session for the Databricks compute environment
from pyspark.sql import SparkSession
from ncaa_tournament_predictor.config import Config
from ncaa_tournament_predictor.databricks import get_databricks_spark_session

# Explicit typing as SparkSession here to help out intellisense...DatabricksSession intellisense
# isn't very good. In all my exploration so far, the DatabricksSession is compatible with the SparkSession
spark: SparkSession = get_databricks_spark_session(Config.databricks_profile())

In [ ]:
# Run all cells above this one to setup your environment

## Schema Setup

Initial steps to create a Databricks schema for holding NCAA mens basketball data

In [4]:
# Create the ncaa_mens_basketball schema
spark.sql("create schema if not exists object_computing.ncaa_mens_basketball;")

""


## Raw Data Volumes
Setup volumes for holding raw data files from various external data sources (CSVs, text files, etc)

In [5]:
# Create a volume for raw Kaggle stats data

from ncaa_tournament_predictor import volumes

raw_kaggle_stats_sql_object = volumes.as_sql_object(volumes.raw_kaggle_stats)
spark.sql(f"create volume if not exists {raw_kaggle_stats_sql_object}")

""


In [6]:
# Copy raw data into the raw_kaggle_stats volume

import os

from ncaa_tournament_predictor import volumes

notebook_dir = os.path.abspath(os.getcwd())
kaggle_dataset_path = os.path.abspath(
    os.path.join(notebook_dir, "../datasets/kaggle_ncaa_stats")
)

for filename in os.listdir(kaggle_dataset_path):
    spark.copyFromLocalToFs(
        local_path=os.path.join(kaggle_dataset_path, filename),
        dest_path=os.path.join(volumes.without_dbfs_protocol(volumes.raw_kaggle_stats), filename)
    )

In [7]:
# Read the Kaggle stats dataset
from ncaa_tournament_predictor import transformation, volumes

raw_kaggle_stats = (
    spark.read.format("csv")
        .options(header=True, inferSchema=True, mergeSchema=True)
        .load(volumes.raw_kaggle_stats)
)
cleaned_ncaa_data = transformation.get_cleaned_kaggle_stats(raw_kaggle_stats)

In [8]:
# Create a volume for raw head-to-head data

from ncaa_tournament_predictor import volumes

spark.sql(f"create volume if not exists {volumes.as_sql_object(volumes.raw_head_to_head)}")

""


In [9]:
# Copy raw data into the raw_head_to_head volume

import os

from ncaa_tournament_predictor import volumes

notebook_dir = os.path.abspath(os.getcwd())
head_to_head_dataset_path = os.path.abspath(
    os.path.join(notebook_dir, "../datasets/kenpom_head_to_head")
)

for filename in os.listdir(head_to_head_dataset_path):
    spark.copyFromLocalToFs(
        local_path=os.path.join(head_to_head_dataset_path, filename),
        dest_path=os.path.join(volumes.without_dbfs_protocol(volumes.raw_head_to_head), filename)
    )

## Data Cleanup & Transformation
Process the raw data, clean it up, and transform it for analysis

In [ ]:
# Create the cleaned Kaggle datasets table
from ncaa_tournament_predictor.jobs import kaggle_stats

kaggle_stats.run_job()

2025/03/27 15:17:13 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://dbc-6ec5c610-07d0.cloud.databricks.com.


2025/03/27 15:17:26 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['team', 'college_season'] of table 'object_computing.ncaa_mens_basketball.game_prediction_features' to NOT NULL.


2025/03/27 15:17:29 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['team', 'college_season'] on table 'object_computing.ncaa_mens_basketball.game_prediction_features'.


MlflowException: Reading Databricks credential configuration failed with MLflow tracking URI 'None'. Please ensure that the 'databricks-sdk' PyPI library is installed, the tracking URI is set correctly, and Databricks authentication is properly configured. The tracking URI can be either 'databricks' (using 'DEFAULT' authentication profile) or 'databricks://{profile}'. You can configure Databricks authentication in several ways, for example by specifying environment variables (e.g. DATABRICKS_HOST + DATABRICKS_TOKEN) or logging in using 'databricks auth login'. 
For details on configuring Databricks authentication, please refer to 'https://docs.databricks.com/en/dev-tools/auth/index.html#unified-auth'.

In [10]:
# Create the cleaned head-to-head table

from ncaa_tournament_predictor.jobs import head_to_head

head_to_head.run_job()

## Creating Training Datasets
Combine data sets to create a dataset used for training an ML model

In [12]:
# Join the 
from pyspark.sql.functions import rand

from ncaa_tournament_predictor import transformation, tables


team_stats = spark.read.table(tables.cleaned_kaggle_stats)
head_to_head_results = spark.read.table(tables.cleaned_head_to_head_results)

training_dataset = transformation.get_training_dataset(team_stats, head_to_head_results)
training_dataset_sample = training_dataset.orderBy(rand()).limit(500)
training_dataset.count()

68312